In [0]:
!pip install -q pyinterval

     |████████████████████████████████| 1.6MB 13.5MB/s 


In [0]:
import numpy as np
import pandas as pd
from interval import interval, inf, imath
from sympy.solvers import solve
from sympy import Symbol

In [0]:
class IntervalMatrix(object):
  @staticmethod
  def from_lists(arrayOfArrays):
    res = IntervalMatrix(len(arrayOfArrays), len(arrayOfArrays[0]))
    res.data = [[interval[bounds[0], bounds[1]] if isinstance(bounds, list) else interval[bounds] for bounds in row] for row in arrayOfArrays]
    return res
  
  def __init__(self, rows, columns):
    self.rows = rows
    self.columns = columns

  def __add__(self, other):
    res = IntervalMatrix(self.rows, self.columns)
    res.data = [[val1 + val2 for val1, val2 in zip(row1, row2)] for row1, row2 in zip(self.data, other.data)]
    return res

  def __sub__(self, other):
    res = IntervalMatrix(self.rows, self.columns)
    res.data = [[val1 - val2 for val1, val2 in zip(row1, row2)] for row1, row2 in zip(self.data, other.data)]
    return res

  def __mul__(self, other):
    if not self.columns == other.rows:
      raise "Nigga you blind or somethin?"
    res = IntervalMatrix(self.rows, other.columns)
    res.data = [[sum([v1 * v2 for v1,v2 in zip(leftRow, rightColumn)]) for rightColumn in other.transpose().data] for leftRow in self.data]
    return res

  def transpose(self):
    res = IntervalMatrix(self.columns, self.rows)
    res.data = [[self.data[j][i] for j in range(self.rows)] for i in range(self.columns)]
    return res

  def __getitem__(self, pos):
    row, column = pos
    return self.data[row][column]

  def __str__(self):
    res = ""
    for row in self.data:
      res += "|\t"
      for val in row:
        res += f"[{val[0].inf}; {val[0].sup}]"
      res += "\t|\n"
    return res

  def flatten(self):
    return [item for row in self.data for item in row]

In [0]:
##@title Алгоритм Мура (для системи 2х2)
#i or intr -> intervals for X and Y
def get_widths(i):
  return [intr[1] - intr[0] for intr in i]
def get_intervals(i):
  return [interval([intr[0], intr[1]]) for intr in i]
def get_middles(i):
  return np.array([[(intr[0] + intr[1]) / 2] for intr in i])
def get_f_mid(f, intr):
  mids = get_middles(intr)
  return np.array([[f[0](mids[0][0], mids[1][0])], [f[1](mids[0][0], mids[1][0])]])
def get_f_prime(f_prime, intervals):
  res = IntervalMatrix(2,2)
  res.data = [[f_prime[0][0](*intervals), f_prime[0][1](*intervals)], 
              [f_prime[1][0](*intervals), f_prime[1][1](*intervals)]]
  return res

def get_f_prime_inverse(f_prime, intervals):
  jakobian = [[f_prime[0][0](*intervals), f_prime[0][1](*intervals)], 
              [f_prime[1][0](*intervals), f_prime[1][1](*intervals)]]
  
  a_lower = np.array([[jakobian[0][0][0].inf, jakobian[0][1][0].inf],
                      [jakobian[1][0][0].inf, jakobian[1][1][0].inf]])
  a_upper = np.array([[jakobian[0][0][0].sup, jakobian[0][1][0].sup],
                      [jakobian[1][0][0].sup, jakobian[1][1][0].sup]])
  b_first_col = np.array([0, 1])
  b_second_col = np.array([1, 0])

  inverse_first_column_lower = np.linalg.solve(a_lower, b_first_col)
  inverse_first_column_upper = np.linalg.solve(a_upper, b_first_col)
  inverse_second_column_lower = np.linalg.solve(a_lower, b_second_col)
  inverse_second_column_upper = np.linalg.solve(a_upper, b_second_col)
  return IntervalMatrix.from_lists([[[inverse_first_column_lower[0], inverse_first_column_upper[0]], [inverse_second_column_lower[0], inverse_second_column_upper[0]]],
                         [[inverse_first_column_lower[1], inverse_first_column_upper[1]], [inverse_second_column_lower[1], inverse_second_column_upper[1]]]])

def init_datasource():
  return ([],[],[],[],[])

def append_datasource(
    xn_intervals,  
    widths,  
    middles,  
    u_intervals,  
    f_prime_matrices, 
    new_xn_intervals,
    new_widths,
    new_middles,
    new_u_intervals,
    new_f_prime_matrices):
  xn_intervals.append(new_xn_intervals)
  widths.append(new_widths)
  middles.append(new_middles)
  u_intervals.append(new_u_intervals)
  f_prime_matrices.append(new_f_prime_matrices)

#f - array of functions (lambdas)
#f_prime - matrix of derivatives of functions in f
#intervals - array of intervals for their respectful variables in the lambdas
def system_moore(f, f_prime, intervals, e):

  xn_intervals, widths, middles, u_intervals, f_prime_matrices = init_datasource()
  add_data = lambda new: append_datasource( xn_intervals, widths, middles, u_intervals, f_prime_matrices, *new)
  
  width = get_widths(intervals)

  if max(width) < e:
    add_data(
        (f"[{intervals[0][0]}; {intervals[0][1]}]\n[{intervals[1][0]}; {intervals[1][1]}]",
        f"ω(Xn) = {width[0]}\nω(Yn) = {width[1]}",
        "",
        "",
        f"Found X = [{intervals[0][0]}; {intervals[0][1]}]\nFound Y = [{intervals[1][0]}; {intervals[1][1]}]"))
    
    return (intervals, widths, middles, u_intervals, f_prime_matrices)

  X0 = get_intervals(intervals)
  mid = get_middles(intervals)
  f_mid = get_f_mid(f, intervals)
  
  if f_mid[0][0] == 0.0 and f_mid[1][0] == 0.0:
    #ділити на 4 сегменти
    #зліва знизу
    left_part_intervals = [[intervals[0][0], mid[0][0]], [intervals[1][0], mid[1][0]]]
    add_data(system_moore(f, f_prime, left_part_intervals, e))
    #справа знизу
    right_part_intervals = [[mid[0][0], intervals[0][1]], [intervals[1][0], mid[1][0]]]
    add_data(system_moore(f, f_prime, right_part_intervals, e))
    #зліва вгорі
    left_part_intervals = [[intervals[0][0], mid[0][0]], [intervals[1][0], mid[1][0]]]
    add_data(system_moore(f, f_prime, left_part_intervals, e))
    #справа вгорі
    right_part_intervals = [[mid[0][0], intervals[0][1]], [mid[1][0], intervals[1][1]]]
    add_data(system_moore(f, f_prime, right_part_intervals, e))

    
  f_derivative_inverse = get_f_prime_inverse(f_prime, X0)
  mid_interval = IntervalMatrix.from_lists([[m] for m in mid])
  f_mid_interval = IntervalMatrix.from_lists([[[f_mid[0][0], f_mid[0][0]]], 
                                       [[f_mid[1][0], f_mid[1][0]]]])

  U = mid_interval - f_derivative_inverse * f_mid_interval
  Xn = [[u & x0] for u, x0 in zip(U.flatten(), X0)]
  add_data(
      (f"Xn = [{intervals[0][0]}; {intervals[0][1]}]\nYn = [{intervals[1][0]}; {intervals[1][1]}]",
      f"ω(Xn) = {width[0]}\nω(Yn) = {width[1]}",
      f"mid Xn = {mid[0][0]}\nmid Yn = {mid[1][0]}",
      f"Ux = [{U[0,0][0].inf}; {U[0,0][0].sup}]\nUy = [{U[1,0][0].inf}; {U[1,0][0].sup}]",
      str(f_derivative_inverse)))
  
  for x in Xn[0]:
    for y in Xn[1]:
      append_datasource(
      xn_intervals, 
      widths, 
      middles, 
      u_intervals, 
      f_prime_matrices, 
      *system_moore(f, f_prime, [[x[0].inf, x[0].sup], [y[0].inf, y[0].sup]], e))
  
  return (intervals, widths, middles, u_intervals, f_prime_matrices)

In [0]:
##@title Алгоритм Мура (задача для системи)
print("Системний алгоритм Мура");
print("f1(x,y) = 3*x - 5*y - 4 = 0");
print("f2(x,y) = x - 4*y + 1 = 0");
print("Корінь: (x, y) = (3, 1)");
print("Інтервали: X = [0; 6], Y = [-3; 3]")
print('-'*80);

e = 10**(-6)

f1 = lambda x, y: 3*x - 5*y - 4
f2 = lambda x, y: x - 4*y + 1

f1_der_x = lambda x, y: -1 - 5*y
f1_der_y = lambda x, y: 3*x - 9
f2_der_x = lambda x, y: 2 - 4*y
f2_der_y = lambda x, y: x - 3

X0 = [0, 6]
Y0 = [-3, 3]

f = np.array([f1, f2])
f_prime = np.array([[f1_der_x, f1_der_y], [f2_der_x, f2_der_y]])
intervals = np.array([X0, Y0])

intervals1, widths, middles, u_intervals, f_prime_matrices = system_moore(f, f_prime, intervals, e)

pd.set_option('display.max_rows', 999)  
df = pd.DataFrame({"Xn": intervals1, "ω(Xn)": widths, "x̅_n": middles, "U": u_intervals, "F'^-1(Xn)": f_prime_matrices})
display(df)

Системний алгоритм Мура
f1(x,y) = 3*x - 5*y - 4 = 0
f2(x,y) = x - 4*y + 1 = 0
Корінь: (x, y) = (3, 1)
Інтервали: X = [0; 6], Y = [-3; 3]
--------------------------------------------------------------------------------


ComponentError: ignored